# 実験を終了する

実験記録をデータガバナンス機能に保存して、実験を終了します。

## 1.実行環境構成を記録する

In [ ]:
%%sh
conda env export -n base > ~/environment.yml
pip freeze > ~/requirements.txt

## 2. 高性能実験環境から実験記録を取得する

高性能実験環境を利用している場合には、以下のセルを実行してください。

In [ ]:
import os
tmp = os.environ["BINDER_REF_URL"]
tmp = tmp.split("/")
tmp = tmp[len(tmp) - 1]
repo_name = tmp[:tmp.find(".")]

!rm ~/input_data/*
!rm ~/output_data/*
!rm ~/source/*

!scp mdx:~/$repo_name/input_data/* ~/input_data/
!scp mdx:~/$repo_name/output_data/* ~/output_data/
!scp mdx:~/$repo_name/source/* ~/source/

## 3. 実験記録をデータガバナンス機能に同期する

実験記録をデータガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob

# Git管理のパスのリストを作成する
%cd ~/
files = os.listdir()
# ディレクトリ一覧からinput_dataとoutput_dataのディレクトリを排除する
dirs = [f for f in files if os.path.isdir(f)]
dirs.remove('input_data')
dirs.remove('output_data')
# HOME直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]
files.extend(dirs)
save_path = files

In [ ]:
import papermill as pm

%cd ~/
annexed_save_path = ['input_data', 'output_data']
# Git-annex管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '実験終了', PATH = annexed_save_path, IS_RECURSIVE = False)
)
# Git管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '実験終了', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
)